In [0]:
%run "../Configuration"

In [0]:
# Define the metadata widget:
dbutils.widgets.text("entityName", "")
dbutils.widgets.text("entityIngestionColumns", "")
dbutils.widgets.text("sourceSystem", "")

# Save parameters to variables:
entityName =       dbutils.widgets.get('entityName')
sourceSystemName = dbutils.widgets.get('sourceSystem')
columns =          json.loads(dbutils.widgets.get('entityIngestionColumns'))
# Query sourceEntity table to find location of raw data:
raw_path =         BRONZE + entity_bronze(entityName)

In [0]:
# Ignore if date:
if entityName == 'date':
    # Currently no date table.  Exit notebook:
    dbutils.notebook.exit('None')

In [0]:
# Create dataframe:
df = spark.read.format('parquet').load(raw_path)

In [0]:
# Cast data types:
df = cast_data_types_transformation(columns, df)

# Drop unwanted columns:
df = drop_unwanted_columns_transformation(columns, df)

# Drop duplicate values:
df = drop_duplicates_transformation(df)
    

In [0]:
# Check data exists:
if df.count() > 0:
    # Set location for delta table:
    file_path = f"/{sourceSystemName}/{entityName}/"
    
    # Write to delta table:
    df.write.mode('overwrite').format('delta').option('header', 'true').save(SILVER + file_path)

    # Output file_path:
    dbutils.notebook.exit(file_path)
else:
    dbutils.notebook.exit(-1)
